In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math
import seaborn as sns
%matplotlib inline

In [3]:
ps = pd.read_csv('c:\users\lisak\documents\github\patchseq_gdoc_analysis\patch_seq_log_mouse.csv')

In [5]:
ps = ps[["Date", "User", "Rig #", "cell type", "Post patch?", "Post patch pipette R", "Nucleus sucked in?", "SM_QC_PF"]]
ps.dropna(subset = ["Post patch?", "SM_QC_PF", "Rig #"], inplace = True)
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["cell type"] == "tdt+"]
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
ps

,Date,User,Rig #,cell type,Post patch?,Post patch pipette R,Nucleus sucked in?,SM_QC_PF
2187,2016-10-20,P1,5,tdt+,Nucleated,NaN,NaN,pass
2362,2016-10-28,P1,7,tdt+,No-Seal,NaN,NaN,fail
2382,2016-10-28,P4,6,tdt+,No-Seal,NaN,NaN,fail
2390,2016-10-31,P1,5,tdt+,No-Seal,NaN,NaN,pass
2391,2016-10-31,P1,5,tdt+,No-Seal,NaN,NaN,fail
2393,2016-10-31,P1,5,tdt+,No-Seal,NaN,NaN,pass
2394,2016-10-31,P1,5,tdt+,No-Seal,NaN,NaN,pass
2395,2016-10-31,P1,5,tdt+,Nucleated,NaN,NaN,pass
2396,2016-10-31,P1,5,tdt+,No-Seal,NaN,NaN,fail
2399,2016-10-31,P2,7,tdt+,No-Seal,NaN,NaN,fail


In [6]:
def get_count(x):    
    count = x["Post patch?"].count()
    return count

def calc_pct(part,total):
    pct = float(get_count(part)) / get_count(total)
    return pct

In [7]:
def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 501))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 500))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | (((z['Post patch?'] == 'nucleus_absent') | (z['Post patch?'] == 'no_nuclues_visible')) & (z['Post patch pipette R'] >= 501))]
    return outside

def no_seal(w): 
    no = w[(w["Post patch?"] == 'No-Seal') | (((w['Post patch?'] == 'nucleus_absent') | (w['Post patch?'] == 'no_nucleus_visible')) & (w['Post patch pipette R'] <= 500))]
    return no
    
def entire_cell(v):
    entire = v[(v["Post patch?"] == 'Entire-Cell') | (v['Post patch?'] == 'entire_cell')]
    return entire
    
def pass_only(df):
    pas = df[df["SM_QC_PF"] == "pass"]
    return pas

def gen_filter(df, col, cond):
    filtr = df[df[col] == cond]
    return filtr

In [8]:
nuc = nucleated(ps)
nuc["post patch class"] = "Nucleated"

part_nuc = partial_nucleated(ps)
part_nuc["post patch class"] = "Partial nucleated"

out_out = outside_out(ps)
out_out["post patch class"] = "Outside out"

noSeal = no_seal(ps)
noSeal["post patch class"] = "No seal"

entirecell = entire_cell(ps)
entirecell["post patch class"] = "Entire Cell"

frames = [nuc, part_nuc, out_out, noSeal, entirecell]
ps2 = pd.concat(frames)
ps2

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anacond

,Date,User,Rig #,cell type,Post patch?,Post patch pipette R,Nucleus sucked in?,SM_QC_PF,post patch class
2187,2016-10-20,P1,5,tdt+,Nucleated,NaN,NaN,pass,Nucleated
2395,2016-10-31,P1,5,tdt+,Nucleated,NaN,NaN,pass,Nucleated
2400,2016-10-31,P2,7,tdt+,Nucleated,NaN,NaN,fail,Nucleated
2402,2016-10-31,P2,7,tdt+,Nucleated,NaN,NaN,pass,Nucleated
2403,2016-10-31,P2,7,tdt+,Nucleated,NaN,NaN,pass,Nucleated
2408,2016-10-31,P4,6,tdt+,Nucleated,NaN,NaN,fail,Nucleated
2409,2016-10-31,P4,6,tdt+,Nucleated,NaN,NaN,pass,Nucleated
2410,2016-10-31,P4,6,tdt+,Nucleated,NaN,NaN,pass,Nucleated
2411,2016-10-31,P4,6,tdt+,Nucleated,NaN,NaN,pass,Nucleated
2431,2016-11-01,P4,6,tdt+,Nucleated,NaN,NaN,pass,Nucleated


In [9]:
ps2 = ps2[["Date", "User", "Rig #", "cell type", "SM_QC_PF", "post patch class"]]
ps2

,Date,User,Rig #,cell type,SM_QC_PF,post patch class
2187,2016-10-20,P1,5,tdt+,pass,Nucleated
2395,2016-10-31,P1,5,tdt+,pass,Nucleated
2400,2016-10-31,P2,7,tdt+,fail,Nucleated
2402,2016-10-31,P2,7,tdt+,pass,Nucleated
2403,2016-10-31,P2,7,tdt+,pass,Nucleated
2408,2016-10-31,P4,6,tdt+,fail,Nucleated
2409,2016-10-31,P4,6,tdt+,pass,Nucleated
2410,2016-10-31,P4,6,tdt+,pass,Nucleated
2411,2016-10-31,P4,6,tdt+,pass,Nucleated
2431,2016-11-01,P4,6,tdt+,pass,Nucleated


In [13]:
ps3 = ps2[ps2["User"] == "P9"]
ps3

,Date,User,Rig #,cell type,SM_QC_PF,post patch class
7193,2017-09-18,P9,3,tdt+,pass,Nucleated
7194,2017-09-18,P9,3,tdt+,pass,Nucleated
7195,2017-09-18,P9,3,tdt+,pass,Nucleated
7196,2017-09-18,P9,3,tdt+,fail,Nucleated
7197,2017-09-18,P9,3,tdt+,pass,Nucleated
7198,2017-09-18,P9,3,tdt+,pass,Nucleated
7199,2017-09-18,P9,3,tdt+,pass,Nucleated
7200,2017-09-18,P9,3,tdt+,pass,Nucleated
7201,2017-09-18,P9,3,tdt+,fail,Nucleated
7202,2017-09-18,P9,3,tdt+,pass,Nucleated


In [14]:
ps4 = ps3[["User", "SM_QC_PF"]]
ps4

,User,SM_QC_PF
7193,P9,pass
7194,P9,pass
7195,P9,pass
7196,P9,fail
7197,P9,pass
7198,P9,pass
7199,P9,pass
7200,P9,pass
7201,P9,fail
7202,P9,pass
